## 准备数据

In [13]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [14]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [15]:
class myModel(tf.keras.Model):
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        super(myModel, self).__init__()
        self.flatten = layers.Flatten()  # 把 28x28 拉直为 784 维
        self.dense1 = layers.Dense(128, activation="relu")
        self.dense2 = layers.Dense(10)  # 10 分类，无激活，返回 logits
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x = self.flatten(x)
        x = self.dense1(x)
        logits = self.dense2(x)
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [16]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = model.trainable_variables
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [19]:
train_data, test_data = mnist_dataset()
for epoch in range(200):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 1.4100593 ; accuracy 0.6946333
epoch 1 : loss 1.4051675 ; accuracy 0.69596666
epoch 2 : loss 1.4002997 ; accuracy 0.69765
epoch 3 : loss 1.3954562 ; accuracy 0.69895
epoch 4 : loss 1.3906373 ; accuracy 0.70005
epoch 5 : loss 1.3858421 ; accuracy 0.70121664
epoch 6 : loss 1.3810716 ; accuracy 0.70248336
epoch 7 : loss 1.3763255 ; accuracy 0.70346665
epoch 8 : loss 1.3716035 ; accuracy 0.7048333
epoch 9 : loss 1.3669056 ; accuracy 0.70595
epoch 10 : loss 1.3622316 ; accuracy 0.7073
epoch 11 : loss 1.3575811 ; accuracy 0.7086167
epoch 12 : loss 1.3529541 ; accuracy 0.7100667
epoch 13 : loss 1.3483514 ; accuracy 0.71098334
epoch 14 : loss 1.3437726 ; accuracy 0.7123167
epoch 15 : loss 1.3392189 ; accuracy 0.7136667
epoch 16 : loss 1.3346896 ; accuracy 0.71461666
epoch 17 : loss 1.3301841 ; accuracy 0.71608335
epoch 18 : loss 1.3257024 ; accuracy 0.7173333
epoch 19 : loss 1.3212456 ; accuracy 0.7183833
epoch 20 : loss 1.3168135 ; accuracy 0.71958333
epoch 21 : loss 1.3124065 